In [ ]:
# first - save cleaned df as csv in other notebook

In [ ]:
instead of zip, grouping should be by worst 100/not
target is number of arrests (or percent) instead of value

In [1]:
#for math and data structure
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error


#for modeling
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller

#for visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('ISR_data_clean.csv',low_memory=False)
df.head()

,CONTACT_CARD_ID,CONTACT_TYPE_CD,CARD_NO,CONTACT_DATE,CONTACT_HOUR,SUBMITTING_BEAT_CD,JUVENILE_I,CLOTHING_DESCR,NAME_VERIFIED_I,RD_NO,...,STOLEN_PROPERTY_I,SEARCH_PROPERTY_I,S_ALCOHOL_I,S_PARA_I,S_STOLEN_PROPERTY_I,GANG_OTHER,OTHER_DESCR,S_OTHER_I,S_OTHER_DESCR,NAME
0,20410357,INVSTG,ISR000410357,01-JAN-2018 00:08,0,2523,N,"BROWN JACKET, JEANS",NaN,JB100012,...,NaN,N,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN
1,20410350,INVSTG,ISR000410350,01-JAN-2018 00:08,0,2523,N,BLUE JEANS AND A BLACK SWEATER,NaN,JB100012,...,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20410351,INVSTG,ISR000410351,01-JAN-2018 00:08,0,2523,N,"GRAY HOODED SWEATER, JEANS",NaN,JB100012,...,NaN,N,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN
3,20410338,INVSTG,ISR000410338,01-JAN-2018 00:10,0,6738,N,"BLACK LONGSLEEVE SHIRT, GREY JEANS",Y,NaN,...,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORCHESTRA ALBANY
4,20410352,INVSTG,ISR000410352,01-JAN-2018 00:30,0,0767,N,"WHITE SOX HAT, BLACK JACKET, BLACK JEANS",Y,NaN,...,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_ts = df[['FO_EMPLOYEE_ID', 'CONTACT_TYPE_CD', 'CONTACT_HOUR', 'CREATED_DATE', 'SEX_CODE_CD', 'RACE_CODE_CD', 'ZIP_CD', 'SEARCH_I', 'BODY_CAMERA_I','ENFORCEMENT_TYPE_CD','INDICATIVE_DRUG_TRANSACTION_I','SUSPICIOUS_OBJECT_I']]
df_ts.head()

,FO_EMPLOYEE_ID,CONTACT_TYPE_CD,CONTACT_HOUR,CREATED_DATE,SEX_CODE_CD,RACE_CODE_CD,ZIP_CD,SEARCH_I,BODY_CAMERA_I,ENFORCEMENT_TYPE_CD,INDICATIVE_DRUG_TRANSACTION_I,SUSPICIOUS_OBJECT_I
0,185285,INVSTG,0,01-JAN-2018 01:46,M,WWH,60641,N,Y,NaN,NaN,NaN
1,675642,INVSTG,0,01-JAN-2018 01:35,M,WWH,60641,N,Y,NaN,NaN,NaN
2,185285,INVSTG,0,01-JAN-2018 01:35,M,WWH,60641,N,Y,NaN,NaN,NaN
3,469481,INVSTG,0,01-JAN-2018 01:01,M,WWH,60634,N,N,NaN,NaN,NaN
4,666758,INVSTG,0,01-JAN-2018 01:36,M,BLK,60621,N,Y,NaN,NaN,Y


In [5]:
df_ts.isna().sum()

FO_EMPLOYEE_ID                        0
CONTACT_TYPE_CD                       0
CONTACT_HOUR                          0
CREATED_DATE                          0
SEX_CODE_CD                           0
RACE_CODE_CD                          0
ZIP_CD                              411
SEARCH_I                              0
BODY_CAMERA_I                         0
ENFORCEMENT_TYPE_CD              222578
INDICATIVE_DRUG_TRANSACTION_I    270898
SUSPICIOUS_OBJECT_I              257093
dtype: int64

In [6]:
df_ts.dropna(subset = ["ZIP_CD"], axis=0, inplace=True)
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289734 entries, 0 to 290144
Data columns (total 12 columns):
FO_EMPLOYEE_ID                   289734 non-null int64
CONTACT_TYPE_CD                  289734 non-null object
CONTACT_HOUR                     289734 non-null int64
CREATED_DATE                     289734 non-null object
SEX_CODE_CD                      289734 non-null object
RACE_CODE_CD                     289734 non-null object
ZIP_CD                           289734 non-null object
SEARCH_I                         289734 non-null object
BODY_CAMERA_I                    289734 non-null object
ENFORCEMENT_TYPE_CD              67459 non-null object
INDICATIVE_DRUG_TRANSACTION_I    19217 non-null object
SUSPICIOUS_OBJECT_I              33002 non-null object
dtypes: int64(2), object(10)
memory usage: 28.7+ MB


In [7]:
# turn enforcement_type_cd into numerical (binary) to indicate arrest

df_ts['ARREST_BINARY'] = np.where(df_ts['ENFORCEMENT_TYPE_CD'] == "ARR", 1, 0)
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289734 entries, 0 to 290144
Data columns (total 13 columns):
FO_EMPLOYEE_ID                   289734 non-null int64
CONTACT_TYPE_CD                  289734 non-null object
CONTACT_HOUR                     289734 non-null int64
CREATED_DATE                     289734 non-null object
SEX_CODE_CD                      289734 non-null object
RACE_CODE_CD                     289734 non-null object
ZIP_CD                           289734 non-null object
SEARCH_I                         289734 non-null object
BODY_CAMERA_I                    289734 non-null object
ENFORCEMENT_TYPE_CD              67459 non-null object
INDICATIVE_DRUG_TRANSACTION_I    19217 non-null object
SUSPICIOUS_OBJECT_I              33002 non-null object
ARREST_BINARY                    289734 non-null int64
dtypes: int64(3), object(10)
memory usage: 30.9+ MB


In [8]:
# add 200 highest officers back in

arrest_pct = df_ts.groupby('FO_EMPLOYEE_ID', as_index=False)['ARREST_BINARY'].mean()
arrest_pct.sort_values(by='ARREST_BINARY', ascending=False).head(10)

,FO_EMPLOYEE_ID,ARREST_BINARY
1565,200661,1.0
5599,706868,1.0
1381,176365,1.0
5764,726578,1.0
5229,662338,1.0
7714,968265,1.0
786,96286,1.0
7437,930217,1.0
1372,175326,1.0
7435,930070,1.0


In [9]:
# limit to officers with more than 20 arrests

filtered = df_ts.groupby('FO_EMPLOYEE_ID').filter(lambda x: len(x) >= 21)
filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256394 entries, 0 to 290144
Data columns (total 13 columns):
FO_EMPLOYEE_ID                   256394 non-null int64
CONTACT_TYPE_CD                  256394 non-null object
CONTACT_HOUR                     256394 non-null int64
CREATED_DATE                     256394 non-null object
SEX_CODE_CD                      256394 non-null object
RACE_CODE_CD                     256394 non-null object
ZIP_CD                           256394 non-null object
SEARCH_I                         256394 non-null object
BODY_CAMERA_I                    256394 non-null object
ENFORCEMENT_TYPE_CD              59723 non-null object
INDICATIVE_DRUG_TRANSACTION_I    17925 non-null object
SUSPICIOUS_OBJECT_I              29346 non-null object
ARREST_BINARY                    256394 non-null int64
dtypes: int64(3), object(10)
memory usage: 27.4+ MB


In [11]:
# and find the officers with the highest arrest percentage

filt_arrest_pct = filtered.groupby('FO_EMPLOYEE_ID', as_index=False)['ARREST_BINARY'].mean()
highest_100 = filt_arrest_pct.sort_values(by='ARREST_BINARY', ascending=False)[:100]

In [13]:
#reset axis

highest_100.reset_index(drop=True, inplace=True)
highest_100.head()

,FO_EMPLOYEE_ID,ARREST_BINARY
0,262036,1.000000
1,4026,0.969697
2,742961,0.965714
3,274229,0.909091
4,936724,0.870130


In [14]:
officer_id_list = highest_100['FO_EMPLOYEE_ID'].tolist()

In [15]:
df_ts['HIGHEST_100_OFFICERS'] = np.isin(df_ts['FO_EMPLOYEE_ID'], officer_id_list)
df_ts.head()

,FO_EMPLOYEE_ID,CONTACT_TYPE_CD,CONTACT_HOUR,CREATED_DATE,SEX_CODE_CD,RACE_CODE_CD,ZIP_CD,SEARCH_I,BODY_CAMERA_I,ENFORCEMENT_TYPE_CD,INDICATIVE_DRUG_TRANSACTION_I,SUSPICIOUS_OBJECT_I,ARREST_BINARY,HIGHEST_100_OFFICERS
0,185285,INVSTG,0,01-JAN-2018 01:46,M,WWH,60641,N,Y,NaN,NaN,NaN,0,False
1,675642,INVSTG,0,01-JAN-2018 01:35,M,WWH,60641,N,Y,NaN,NaN,NaN,0,False
2,185285,INVSTG,0,01-JAN-2018 01:35,M,WWH,60641,N,Y,NaN,NaN,NaN,0,False
3,469481,INVSTG,0,01-JAN-2018 01:01,M,WWH,60634,N,N,NaN,NaN,NaN,0,False
4,666758,INVSTG,0,01-JAN-2018 01:36,M,BLK,60621,N,Y,NaN,NaN,Y,0,False


In [23]:
# take care of missing info

df_ts['ENFORCEMENT_TYPE_CD'] = df_ts['ENFORCEMENT_TYPE_CD'].fillna(0)
df_ts['ENFORCEMENT_TYPE_CD'].value_counts()

0       222275
ARR      34928
OTH      17073
ANOV     12326
PSC       3132
Name: ENFORCEMENT_TYPE_CD, dtype: int64

In [24]:
df_ts['INDICATIVE_DRUG_TRANSACTION_I'] = df_ts['INDICATIVE_DRUG_TRANSACTION_I'].fillna(0)
df_ts['INDICATIVE_DRUG_TRANSACTION_I'].value_counts()

0    270517
Y     17816
N      1401
Name: INDICATIVE_DRUG_TRANSACTION_I, dtype: int64

In [25]:
df_ts['SUSPICIOUS_OBJECT_I'] = df_ts['SUSPICIOUS_OBJECT_I'].fillna(0)
df_ts['SUSPICIOUS_OBJECT_I'].value_counts()

0    256732
Y     30532
N      2470
Name: SUSPICIOUS_OBJECT_I, dtype: int64

In [33]:
df_ts.head()

,FO_EMPLOYEE_ID,CONTACT_TYPE_CD,CONTACT_HOUR,CREATED_DATE,SEX_CODE_CD,RACE_CODE_CD,ZIP_CD,SEARCH_I,BODY_CAMERA_I,ENFORCEMENT_TYPE_CD,INDICATIVE_DRUG_TRANSACTION_I,SUSPICIOUS_OBJECT_I,ARREST_BINARY,HIGHEST_100_OFFICERS
0,185285,INVSTG,0,01-JAN-2018 01:46,M,WWH,60641,N,Y,0,0,0,0,False
1,675642,INVSTG,0,01-JAN-2018 01:35,M,WWH,60641,N,Y,0,0,0,0,False
2,185285,INVSTG,0,01-JAN-2018 01:35,M,WWH,60641,N,Y,0,0,0,0,False
3,469481,INVSTG,0,01-JAN-2018 01:01,M,WWH,60634,N,N,0,0,0,0,False
4,666758,INVSTG,0,01-JAN-2018 01:36,M,BLK,60621,N,Y,0,0,Y,0,False


In [31]:
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289734 entries, 0 to 290144
Data columns (total 14 columns):
FO_EMPLOYEE_ID                   289734 non-null int64
CONTACT_TYPE_CD                  289734 non-null object
CONTACT_HOUR                     289734 non-null int64
CREATED_DATE                     289734 non-null object
SEX_CODE_CD                      289734 non-null object
RACE_CODE_CD                     289734 non-null object
ZIP_CD                           289734 non-null object
SEARCH_I                         289734 non-null object
BODY_CAMERA_I                    289734 non-null object
ENFORCEMENT_TYPE_CD              289734 non-null object
INDICATIVE_DRUG_TRANSACTION_I    289734 non-null object
SUSPICIOUS_OBJECT_I              289734 non-null object
ARREST_BINARY                    289734 non-null int64
HIGHEST_100_OFFICERS             289734 non-null bool
dtypes: bool(1), int64(3), object(10)
memory usage: 31.2+ MB


In [ ]:
# EDA

In [ ]:
# change datetime to monthly

In [ ]:
df_ts.groupby(["CREATED_DATE","RACE_CODE_CD"]).size().unstack().plot(kind='bar', stacked=True, figsize=(12,6),fontsize=14) 

In [35]:
# onto the time series analysis

In [ ]:
# create new df with just 